# 1. Load Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle


# 2. Function Set

In [2]:
def construct_trans(num_action,num_state):
    trans=[]
    for u in range(num_action):
        p1=[]
        for i in range(num_state):
            p2=[]
            for j in range(num_state):
                p2.append(0)
            p1.append(p2)
        trans.append(p1)

    for u in range(num_action):
        for i in range(num_state):
            trans[u][i][env.step(i,u)[1]]=1

    return trans

        

    

In [3]:
def inteprete_state(state):
    cell=env.idx2cell[int(state/8)] 
    flag=env.num2flag(state%8)
    
    return (cell[0],cell[1],flag)

In [4]:
def VI(num_state,num_action,ACTMAP,trans,rou,idx2cell,goal_pos):
    Q,V,miu=[],[],[]
    #initialize
    Q.append([])
    miu.append([])
    V.append([0]*num_state)

    #iteration begin
    for k in range (1,N):
        Q_k=[]
        for i in range (num_state):
            Q_ki=[]
            for u in range(num_action):
                Q_kiu=0
                for slipcondition in range(2):
                        if slipcondition==0:
                            a=u
                        else:
                            a=ACTMAP[u]
                        add=0
                        for j in range(num_state):
                            add+=trans[a][i][j]*V[k-1][j]

                        if env.step(i,a)[2]==False: #not reach target cell
                            cc=env.step(i,a)[0]+rou*add
                        else: 
                            cc=env.step(i,a)[0]
                        Q_kiu+=prob[slipcondition]*cc
                Q_ki.append(Q_kiu)
            Q_k.append(Q_ki)
        Q.append(Q_k)

        V_k,miu_k=[],[]
        for i in range(num_state):
            V_k.append(max(Q[k][i]))
            miu_k.append(Q[k][i].index(max(Q[k][i])))

        V.append(V_k)
        miu.append(miu_k)

    policy_VI=miu[-1]
    Q_VI=Q[-1]
    return policy_VI,Q_VI

# 3. Value Iteration

In [5]:
#maze environment
from maze_VI import *
env=Maze()

In [6]:
#parameter
num_state=112
num_action=4
slip=0.1
rou=0.9
#initialize
state=env.reset()
trans=construct_trans(num_action,num_state)
prob=[1-slip,slip]
N=1000
idx2cell = {0: (0, 0), 1: (1, 0), 2: (3, 0), 3: (1, 1), 4: (2, 1), 5: (3, 1),6: (0, 2), 7: (1, 2), 8: (2, 2), 9: (1, 3), 10: (2, 3), 11: (3, 3), 12: (0, 4), 13: (1, 4)}
goal_pos = (0,4)

#value iteration for optimal policy
policy_VI,Q_VI=VI(num_state,num_action,ACTMAP,trans,rou,idx2cell,goal_pos)  





In [7]:
#plot cell position and flag vector
record=[]
i=env.reset()
record.append(inteprete_state(i))
for k in range(100):
    r,i,done=env.step(i,policy_VI[i])
    record.append(inteprete_state(i))
    if done==True:
        break
record

[(0, 0, [0, 0, 0]),
 (1, 0, [0, 0, 0]),
 (1, 1, [0, 0, 0]),
 (1, 2, [0, 0, 0]),
 (0, 2, [1, 0, 0]),
 (1, 2, [1, 0, 0]),
 (1, 3, [1, 0, 0]),
 (2, 3, [1, 0, 0]),
 (3, 3, [1, 0, 1]),
 (2, 3, [1, 0, 1]),
 (1, 3, [1, 0, 1]),
 (1, 4, [1, 0, 1]),
 (0, 4, [1, 0, 1])]

In [8]:
#save
f = open('store.pckl','wb')
pickle.dump(Q_VI, f)
pickle.dump(policy_VI,f)
f.close()